# [LintCode](https://www.lintcode.com/problem/) Problems

## 1. A+B Problem (Easy)

**Description**

Write a function that add two numbers A and B. You should not use + or any arithmetic operators.

**Clarification**

Are a and b both 32-bit integers?

+ **Yes**.

Can I use bit operation?

+ **Sure you can**.

**Example**

Given a=1 and b=2 return 3.

**Challenge**

Of course you can just return a + b to get accepted. But Can you challenge not do it like that?

<u>**Solution**</u>

用**位运算**实现加法, 即像计算机用二进制实现运算一样.

+ 迭代版本

In [1]:
def aplusb(a, b):
    res = a ^ b
    carry = a & b
    while carry:
        temp_a = res # 表示无进位的加法
        temp_b = carry << 1 # 表示进位
        res = temp_a ^ temp_b
        carry = temp_a & temp_b
    return res

In [2]:
%time aplusb(1, 2)

CPU times: user 8 µs, sys: 5 µs, total: 13 µs
Wall time: 16.9 µs


3

+ 递归版本

In [3]:
def aplusb(a, b):
    if a == 0:
        return b
    res = a ^ b # 表示无进位的加法
    carry = (a & b) << 1 # 表示进位
    return aplusb(carry, res)

In [4]:
%time aplusb(1, 2)

CPU times: user 6 µs, sys: 1e+03 ns, total: 7 µs
Wall time: 11 µs


3

## 2. Trailing Zeros (Easy)

**Description**

Write an algorithm which computes the number of trailing zeros in n factorial.

**Example**

11! = 39916800, so the out should be 2

**Challenge**

O(log N) time

<u>**Solution**</u>

+ 直接计算阶乘后统计末尾0的个数可能会<u>**溢出**</u>

+ 从"哪些数相乘可以得到10"这样的角度考虑: 如果将N阶乘表示成K和10的M次方的乘积, 那么N!末尾就有M个0. 如果将N阶乘分解成素数幂次的乘积, 即2的X次方, 3的Y次方, 5的Z次方, ... 的乘积，由于10 = 2 * 5, 所以M只能和X和Z有关, 每一对2和5相乘就可以得到一个10, 于是M = MIN(X, Z), 不难看出X大于Z, 因为被2整除的频率要比被5整除的频率高得多, 所以可以把公式简化为M=Z. 由上面的分析可以看出, 只要计算Z的值，就可以得到N!末尾0的个数.

+ **方法1**: 直接求出N阶乘的所有因式(1,2,3,...,N)分解中5的指数, 然后求和

In [5]:
def trailingZeros(n):
    num = 0
    for i in range(5, n + 1, 5):
        j = i
        while j % 5 == 0:
            num += 1
            j /= 5
    return num

In [6]:
%time trailingZeros(11)

CPU times: user 9 µs, sys: 1e+03 ns, total: 10 µs
Wall time: 12.9 µs


2

+ **方法2**

Z = N / 5 + N / (5 \* 5) + N / (5 \* 5 \* 5) + ... 直到N/(5的K次方)等于0, 公式中 N/5表示不大于N的数中能被5整除的数贡献一个5, N / (5 * 5)表示不大于N的数中能被25整除的数再共享一个5...

In [7]:
def trailingZeros(n):
    num = 0
    while n != 0:
        num += n / 5
        n = n / 5
    return num

In [8]:
%time trailingZeros(11)

CPU times: user 8 µs, sys: 2 µs, total: 10 µs
Wall time: 16.9 µs


2

## 3. Digit Counts (Medium)

**Description**

Count the number of k's between 0 and n. k can be 0 - 9.

**Example**

if n = 12, k = 1 in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

we have FIVE 1's (1, 10, 11, 12)

<u>**Solution**</u>


+ **方法1**: Brute Force(暴力法), 即遍历一遍0~n, 统计每个数字中k出现的个数.

In [9]:
# 检查单个数字num中k的个数
def count(num, k):
    tmp = str(num)
    res = 0
    for i in range(len(tmp)):
        if int(tmp[i]) == k:
            res += 1
    return res

# 总计
def digitCounts(k, n):
    res = 0
    for i in range(n + 1):
        res += count(i, k)
    return res

In [10]:
%time digitCounts(1, 12)

CPU times: user 58 µs, sys: 13 µs, total: 71 µs
Wall time: 64.1 µs


5

+ **方法2**: 直接统计从0-n中, 每一位中可能出现为k的个数. 

举个例子, 假设n是一个5位数, 我们就是要统计从第一位到第五位中, k可能出现的个数. 

更具体一些, 假设n=12345, k=3, m为当前统计到的位数(m=1,2,3,4,5), k出现的个数为cnt, 如果我们要统计百位上3出现的次数, 显然:

1. 小于3*100的数, 我们不予考虑 
2. 紧接我们考虑300~399的数 
3. 再考虑1300~1399, 2300~2399, …, 9300~9399(共100*10=1000个数) 
4. 最后考虑10300~10399, 11300~11399, 12300~12345中3出现个数(共100*2+46=246个数)

从上面看到, 只需固定前3位为300~399, 直接算出后面位数所有可能出现的情况, 就是我们对于当前位数出现的3的个数的统计. 这里, 就是1000+246=1246个数，恰好是 1 + 12345 % 100 + 12345 / (100 \* 10) \* 100.

这里作一些变量设定,

base = 100

lower= 12345 % base

higher = 12345 / (base * 10)

curBit = 12345 % (base * 10) / base

count = 1 + base * higher + lower

则我们得到了计算当k=curBit时, k的个数(count)的大小.

上面的变量表示如下：

1. higer:表示更高位, 如当前位数为3, 则更高位是第4, 5位;
2. lower:表示更低位, 如1, 2位;
3. curBit表示第3位的数字.

我们可以类比分析出:

1. 当k < curBit, 比如求第2位时, 有 count = base * (higer + 1)
2. 当k > curBit, 比如求第4位时, 有 count = base * higer

这里有两个特别需要注意的地方： 

1. 如果k = n = 0, 则count = 1 
2. 如果k = 0, n != 0, 则我们应该不算最高位的情况, 因为最高位不可能为0.

In [11]:
def digitCounts(k, n):
    count = 0
    base = 1
    flag = 1
    if k ==0 and n == 0:
        return 1
    if k == 0:
        flag = 10
    while n / base >= flag:
        curBit = n % (base * 10) / base
        higer = n / (base * 10)
        lower = n % base
        if curBit < k:
            count += base * higer
        elif curBit == k:
            count += 1 + base * higer + lower
        else:
            count += base * (higer + 1)
        base *= 10
    return count

In [12]:
%time digitCounts(1, 12)

CPU times: user 15 µs, sys: 5 µs, total: 20 µs
Wall time: 22.9 µs


5

## 4. Ugly Number II (Medium)

**Description**

Ugly number is a number that only have factors 2, 3 and 5.

Design an algorithm to find the nth ugly number. The first 10 ugly numbers are 1, 2, 3, 4, 5, 6, 8, 9, 10, 12 ...

ⓘ Note that 1 is typically treated as an ugly number.

**Example**

If n=9, return 10.

**Challenge**

O(n log n) or O(n) time.

<u>**Solution**</u>


+ **方法1**: Brute Force(暴力法), 即首先除2, 直到不能整除为止, 然后除5到不能整除为止, 然后除3直到不能整除为止. 最终判断剩余的数字是否为1, 如果是1则为ugly number, 否则不是ugly number.

In [13]:
def nthUglyNumber(n):
    countN = 0
    res = 0
    while countN < n:
        res += 1
        number = res
        while number % 2 == 0:
            number = number / 2
        while number % 3 == 0:
            number = number / 3
        while number % 5 == 0:
            number = number / 5
        if number == 1:
            countN += 1
    return res

In [14]:
%time nthUglyNumber(9)

CPU times: user 23 µs, sys: 11 µs, total: 34 µs
Wall time: 32.9 µs


10

+ **方法2**: 直接寻找ugly number. 由ugly number的定义可知, 任何一个ugly number都是2<sup>i</sup> \* 3<sup>j</sup> \* 5<sup>k</sup>这种形式, 因此不断寻找ugly number, 将它们按从小到大的顺序进行排列, 直到第n个即为结果.

首先定义一个数组存放ugly number, 默认假定1是ugly number, 所以初始化数组ugly[0] = 1, 然后从2, 3, 5这三个种子中挑选, 选择ugly[0] \* 2, ugly[0] \* 3, ugly[0] \* 5中最小的数为新的ugly number, 显然应该选择2, ugly[1] = 2, 然后再从2, 3, 5这三个种子中选择, 这时应该是从ugly[1] \* 2, ugly[0] \* 3, ugly[0] \* 5中进行选择, 显然应该选择3, 即ugly[2] = 3, 然后再从ugly[1] \* 2, ugly[1] \* 3, ugly[0] \* 5中选择最小的, 选择4, 即ugly[3] = 4，依次进行类似操作，从而得到最终的结果。

In [15]:
def nthUglyNumber(n):
    ugly = [0] * n
    ugly[0] = 1
    p_2 = 0
    p_3 = 0
    p_5 = 0
    for i in range(1, n):
        ugly[i] = min(ugly[p_2] * 2, ugly[p_3] * 3, ugly[p_5] * 5)
        if ugly[i] / ugly[p_2] == 2:
            p_2 += 1
        if ugly[i] / ugly[p_3] == 3:
            p_3 += 1
        if ugly[i] / ugly[p_5] == 5:
            p_5 += 1
    return ugly[n - 1]

In [16]:
%time nthUglyNumber(9)

CPU times: user 31 µs, sys: 11 µs, total: 42 µs
Wall time: 37.9 µs


10

## 5. Kth Largest Element (Medium)

**Description**

Find K-th largest element in an array.

ⓘ You can swap elements in the array

**Example**

In array [9,3,2,4,8], the 3rd largest element is 4.

In array [1,2,3,4,5], the 1st largest element is 5, 2nd largest element is 4, 3rd largest element is 3 and etc.

**Challenge**

O(n) time, O(1) extra memory.

<u>**Solution**</u>

The easiest solution is to choose a random pivot, which yields almost certain linear time. Deterministically, one can use median-of-3 pivot strategy (as in the quicksort), which yields linear performance on partially sorted data, as is common in the real world. However, contrived sequences can still cause worst-case complexity; David Musser describes a "median-of-3 killer" sequence that allows an attack against that strategy, which was one motivation for his introselect algorithm.

[Quickselect](https://en.wikipedia.org/wiki/Quickselect) uses the same overall approach as quicksort, choosing one element as a pivot and partitioning the data in two based on the pivot, accordingly as less than or greater than the pivot. However, instead of recursing into both sides, as in quicksort, quickselect only recurses into one side – the side with the element it is searching for. This reduces the average complexity from O(n log n) (in quicksort) to O(n) (in quickselect).

In [17]:
import numpy as np

In [18]:
# 下面程序实现了kth小元素查找
def partition(A, left, right, pivotIndex):
    pivotValue = A[pivotIndex]
    # Move pivot to end
    A[pivotIndex], A[right] = A[right], A[pivotIndex]
    storeIndex = left
    for i in range(left, right):
        if A[i] < pivotValue:
            A[storeIndex], A[i] = A[i], A[storeIndex]
            storeIndex += 1
    # Move pivot to its final place
    A[right], A[storeIndex] = A[storeIndex], A[right]
    return storeIndex

# k's values start with 1
def quickSelect(A, left, right, k):
    while True:
        if left == right:
            return A[left]
        pivotIndex = np.random.randint(left, right + 1)
        pivotIndex = partition(A, left, right, pivotIndex)
        if k - 1 == pivotIndex:
            return A[pivotIndex]
        elif k - 1 < pivotIndex:
            right = pivotIndex - 1
        else:
            left = pivotIndex + 1

In [19]:
A = [9, 3, 2, 4, 8]
k = 1
%time quickSelect(A, 0, len(A) - 1, k)

CPU times: user 64 µs, sys: 150 µs, total: 214 µs
Wall time: 1.13 ms


2

In [20]:
# 下面程序实现了kth大元素查找
def partition(A, left, right, pivotIndex):
    pivotValue = A[pivotIndex]
    # Move pivot to end
    A[pivotIndex], A[right] = A[right], A[pivotIndex]
    storeIndex = left
    for i in range(left, right):
        if A[i] > pivotValue:
            A[storeIndex], A[i] = A[i], A[storeIndex]
            storeIndex += 1
    # Move pivot to its final place
    A[right], A[storeIndex] = A[storeIndex], A[right]
    return storeIndex

# k's values start with 1
def quickSelect(A, left, right, k):
    while True:
        if left == right:
            return A[left]
        pivotIndex = np.random.randint(left, right + 1)
        pivotIndex = partition(A, left, right, pivotIndex)
        if k - 1 == pivotIndex:
            return A[pivotIndex]
        elif k - 1 < pivotIndex:
            right = pivotIndex - 1
        else:
            left = pivotIndex + 1

In [21]:
A = [9, 3, 2, 4, 8]
k = 1
%time quickSelect(A, 0, len(A) - 1, k)

CPU times: user 36 µs, sys: 9 µs, total: 45 µs
Wall time: 42 µs


9

## 6. Merge Two Sorted Arrays (Easy)

**Description**

Merge two given sorted integer array A and B into a new sorted integer array.

**Example**

A=[1,2,3,4]

B=[2,4,5,6]

return [1,2,2,3,4,4,5,6]

**Challenge**

How can you optimize your algorithm if one array is very large and the other is very small?

<u>**Solution**</u>


+ **方法1**: 混合插入有序数组, 由于两个数组都是有序的, 所有只要按顺序比较大小即可. 最先想到的方法是建立一个大小为m+n的新数组, 然后逐个从A和B数组中取出元素比较, 把较小的加入新数组, 然后再考虑A数组有剩余和B数组有剩余这两种情况即可.

In [22]:
def mergeSortedArray(A, B):
    m = len(A)
    n = len(B)
    p_a = 0
    p_b = 0
    C = []
    for i in range(m + n):
        if p_a < m and p_b < n:
            if A[p_a] < B[p_b]:
                C.append(A[p_a])
                p_a += 1
            else:
                C.append(B[p_b])
                p_b += 1
        elif p_a < m and p_b >= n:
            C.append(A[p_a])
            p_a += 1
        elif p_a >= m and p_b < n:
            C.append(B[p_b])
            p_b += 1
    return C

In [23]:
A=[1,2,3,4]
B=[2,4,5,6]
%time mergeSortedArray(A, B)

CPU times: user 15 µs, sys: 5 µs, total: 20 µs
Wall time: 21.9 µs


[1, 2, 2, 3, 4, 4, 5, 6]

+ **方法2**: 上面的方法固然没错, 但还有更简洁的方法, 而且不用申请新变量. 算法思想是: 由于合并后A数组的大小必定是m+n, 所以从最后面开始往前赋值, 先比较A和B中最后一个元素的大小, 把较大的那个插入到m+n-1的位置上, 再依次向前推. 如果A中所有的元素都比B小, 那么前m个还是A原来的内容, 没有改变. 如果A中的数比B大, 当A循环完了, B中还有元素没加入A, 直接用个循环把B中所有的元素覆盖到A剩下的位置.

In [24]:
def mergeSortedArray(A, B):
    m = len(A)
    n = len(B)
    C = [0] * (m + n)
    count = m + n - 1
    m -= 1
    n -= 1
    while m >= 0 and n >= 0:
        if A[m] > B[n]:
            C[count] = A[m]
            count -= 1
            m -= 1
        else:
            C[count] = B[n]
            count -= 1
            n -= 1
    while m >= 0 and n < 0:
        C[count] = A[m]
        count -= 1
        m -= 1
    while n >= 0 and m < 0:
        C[count] = B[n]
        count -= 1
        n -= 1
        
    return C

In [25]:
A=[1,2,3,4]
B=[2,4,5,6]
%time mergeSortedArray(A, B)

CPU times: user 15 µs, sys: 7 µs, total: 22 µs
Wall time: 22.9 µs


[1, 2, 2, 3, 4, 4, 5, 6]

**Note**: 这里看, 我感觉python下实现两种方法并没有太大不同, 一个是从前面开始按从小到大比较排列, 一个是从后面开始按从大到小比较排列.

## 7. Serialize and Deserialize Binary Tree (Medium)

**Description**

Design an algorithm and write code to serialize and deserialize a binary tree. Writing the tree to a file is called 'serialization' and reading back from the file to reconstruct the exact same binary tree is 'deserialization'.

ⓘ There is no limit of how you deserialize or serialize a binary tree, LintCode will take your output of serialize as the input of deserialize, it won't check the result of serialize.

**Example**

An example of testdata: Binary tree {3,9,20,#,#,15,7}, denote the following structure:

 $$\begin{align}3  \\ 
 \diagup \qquad \diagdown \end{align} \\ 
 9 \qquad 20 \\ 
 \qquad \qquad \diagup \qquad \diagdown \\
 \qquad \qquad 15 \qquad 7$$
 
Our data serialization use bfs traversal. This is just for when you got wrong answer and want to debug the input.

You can use other method to do serializaiton and deserialization.

<u>**Solution**</u>

序列化就是将一个数据结构或物体转化为一个位序列, 可以存进一个文件或者内存缓冲器中, 然后通过网络连接在相同或不同的电脑环境中被还原, 还原的过程叫做去序列化.

+ **方法1**: 先序遍历的递归解法. 简单易懂, 我们需要接入输入和输出字符串流istringstream和ostringstream, 对于序列化, 我们从根节点开始，如果节点存在, 则将值存入输出字符串流, 然后分别对其左右子节点递归调用序列化函数即可. 对于去序列化, 我们先读入第一个字符, 以此生成一个根节点, 然后再对根节点的左右子节点递归调用去序列化函数即可.

In [26]:
# Definition of TreeNode
class TreeNode:
    def __init__(self, val):
        self.val = val
        self.left = None
        self.right = None

def serialize(root):
    # Encodes a tree to a single string.
    if root is None:
        return ''
    tree = [root]
    s = []
    while len(tree) > 0:
        node = tree.pop(0)
        if node is None:
            s.append('#')
        else:
            s.append(str(node.val))
            tree.append(node.left)
            tree.append(node.right)
    res = s
    end = len(res) - 1
    while True:
        if res[end] == '#':
            res = res[:-1]
            end -= 1
        else:
            break
    res = ','.join(res)
    s = ','.join(s)
    return res, s
            
def deserialize(data):
    # Decodes your encoded data to tree.
    s = data.split(',')
    i = 0
    if s[0] == '#':
        return None
    root = TreeNode(int(s[0]))
    tree = [root]
    
    while len(tree) > 0:
        node = tree.pop(0)
        # 如果节点不是'#', 则计数器加2
        i += 1
        if s[i] != '#':
            left = TreeNode(int(s[i]))
            node.left = left
            tree.append(left)
        i += 1
        if s[i] != '#':
            right = TreeNode(int(s[i]))
            node.right = right
            tree.append(right)
    return root

In [27]:
root = TreeNode(3)
r_l = TreeNode(9)
r_r = TreeNode(20)
root.left = r_l
root.right = r_r
r_r_l = TreeNode(15)
r_r_r = TreeNode(7)
r_r.left = r_r_l
r_r.right = r_r_r

In [28]:
%time result = serialize(root)
res = result[0]
data = result[1]
print res
%time root = deserialize(data)
print root

CPU times: user 36 µs, sys: 9 µs, total: 45 µs
Wall time: 41 µs
3,9,20,#,#,15,7
CPU times: user 59 µs, sys: 56 µs, total: 115 µs
Wall time: 94.9 µs
<__main__.TreeNode instance at 0x10fb65128>


+ **方法2**: 层序遍历的非递归解法. 这种方法略微复杂一些, 我们需要借助queue来做, 本质是BFS算法, 也不是很难理解, 就是BFS算法的常规套路稍作修改即可.

In [29]:
from Queue import Queue

In [30]:
# Definition of TreeNode
class TreeNode:
    def __init__(self, val):
        self.val = val
        self.left = None
        self.right = None

def serialize(root):
    # Encodes a tree to a single string.
    q = Queue()
    if root is not None:
        q.put(root)
    s = []
    while not q.empty():
        node = q.get()
        if node is None:
            s.append('#')
        else:
            s.append(str(node.val))
            q.put(node.left)
            q.put(node.right)
    res = s
    end = len(res) - 1
    while True:
        if res[end] == '#':
            res = res[:-1]
            end -= 1
        else:
            break
    res = ','.join(res)
    s = ','.join(s)
    return res, s
            
def deserialize(data):
    # Decodes your encoded data to tree.
    s = data.split(',')
    i = 0
    if s[0] == '#':
        return None
    root = TreeNode(int(s[0]))
    q = Queue()
    q.put(root)
    # 使用队列循环节点, 然后有一个计数器, 如果节点不是'#', 则计数器加2
    while q.qsize() > 0:
        node = q.get()
        i += 1
        if s[i] != '#':
            left = TreeNode(int(s[i]))
            node.left = left
            q.put(left)
        i += 1
        if s[i] != '#': 
            right = TreeNode(int(s[i]))
            node.right = right 
            q.put(right) 
        return root 

In [31]:
root = TreeNode(3)
r_l = TreeNode(9)
r_r = TreeNode(20)
root.left = r_l
root.right = r_r
r_r_l = TreeNode(15)
r_r_r = TreeNode(7)
r_r.left = r_r_l
r_r.right = r_r_r

In [32]:
%time result = serialize(root)
res = result[0]
data = result[1]
print res
%time root = deserialize(data)
print root

CPU times: user 622 µs, sys: 497 µs, total: 1.12 ms
Wall time: 638 µs
3,9,20,#,#,15,7
CPU times: user 232 µs, sys: 99 µs, total: 331 µs
Wall time: 249 µs
<__main__.TreeNode instance at 0x10f40d488>


## 8. Rotate String (Easy)

**Description**

Given a string and an offset, rotate string by offset. (rotate from left to right)

**Example**

Given "abcdefg".

offset=0 => "abcdefg"
offset=1 => "gabcdef"
offset=2 => "fgabcde"
offset=3 => "efgabcd"

**Challenge**

Rotate in-place with O(1) extra memory.

<u>**Solution**</u>

+ **方法1**: 三步翻转法

1 题目中传递的参数是string类型, 所以删除/插入元素比一般数组方便, 降低题目的难度, 但也需要知道string类的一些常用成员函数.

2 思路很简单: 将末尾元素复制取出来插入到第一个元素前面, 然后删除末尾元素. 但这里有一个小技巧: 当str循环移动了自身长度次时, str恢复到未做任何移动的初始状态.

例如: str = "abc", 此时str.length() = 3. 第一次移动后, str = "cab", 第二次移动后, str = "bca", 第三次移动后, str = "abc".

3 对于输入参数异常的情况, 要在一开始就处理好.

In [33]:
def rotateString(s, offset):
    if s == '':
        return ''
    if offset == 0:
        return s
    for i in range(1, offset % len(s) + 1):
        s = s[-1] + s[0:-1]
    return s    

In [34]:
s = 'abcdefg'
offset = 2
%time rotateString(s, offset)

CPU times: user 10 µs, sys: 3 µs, total: 13 µs
Wall time: 16.9 µs


'fgabcde'

+ **方法2**: 两步翻转法

In [35]:
def reverse(s, start, end):
    i = start
    j = end
    while i < j:
        s = s[:i] + s[j] + s[i+1:j] + s[i] + s[j+1:]
        i += 1
        j -= 1
    return s

def rotateString(s, offset):
    if s == '':
        return s
    offset = offset % len(s)
    s = reverse(s, 0, len(s) - offset - 1)
    s = reverse(s, len(s) - offset, len(s) - 1)
    s = reverse(s, 0, len(s) - 1)
    return s

In [36]:
s = 'abcdefg'
offset = 2
%time rotateString(s, offset)

CPU times: user 19 µs, sys: 4 µs, total: 23 µs
Wall time: 23.8 µs


'fgabcde'

## 9. Fizz Buzz (Easy)

**Description**

Given number n. Print number from 1 to n. But:

when number is divided by 3, print "fizz".

when number is divided by 5, print "buzz".

when number is divided by both 3 and 5, print "fizz buzz".

**Example**

If n = 15, you should return:

[

  "1", "2", "fizz",
  
  "4", "buzz", "fizz",
  
  "7", "8", "fizz",
  
  "buzz", "11", "fizz",
  
  "13", "14", "fizz buzz"

]

**Challenge**

Can you do it with only one if statement?

<u>**Solution**</u>

题目本身并不难，只是不知道什么才算只有一个if语句.

In [37]:
def fizzBuzz(n):
    while n < 1:
        n = input("n should be larger than 0")
    res = []
    for i in range(1, n + 1):
        if i % 3 == 0 and i % 5 == 0:
            res.append('fizz buzz')
        elif i % 3 == 0:
            res.append('fizz')
        elif i % 5 == 0:
            res.append('buzz')
        else:
            res.append(str(i))
    return res

In [38]:
%time fizzBuzz(15)

CPU times: user 27 µs, sys: 9 µs, total: 36 µs
Wall time: 33.9 µs


['1',
 '2',
 'fizz',
 '4',
 'buzz',
 'fizz',
 '7',
 '8',
 'fizz',
 'buzz',
 '11',
 'fizz',
 '13',
 '14',
 'fizz buzz']